In [37]:
import pandas as pd

In [38]:
file_path = "result.json"

In [39]:
data_df = pd.read_json(file_path)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data_id      45 non-null     object
 1   page_number  45 non-null     int64 
 2   region_type  45 non-null     object
 3   content      45 non-null     object
 4   meta         45 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.9+ KB


In [41]:
# 도표 데이터를 자연어 문장으로 변환하는 함수
def process_chart(data):
    # 내용만 불러와서 "\n"에 따라 분리
    contents = data.content.split("\n")

    # 값 추출 (연도 | 값 형태)
    values = [line.split("|") for line in contents if "|" in line]

    # 데이터 변환
    text_description = "대한민국 명목 국내 총 생산 연도별 값은 다음과 같다: "
    for year, value in values[1:]:
        text_description += f"{year.strip()}년에는 {value.strip()}(억)달러, "

    text_description = text_description.rstrip(", ") + "이다."

    return text_description

# 도표 데이터 변환 실행
data_df['content'] = data_df.apply(lambda x : process_chart(x) if (x.region_type == "도표") and (x.page_number != 1) else x.content, axis = 1)
data_df['region_type'] = data_df.apply(lambda x : "평문" if (x.region_type == "도표") and (x.page_number != 1) else x.region_type, axis = 1)

In [42]:
# 오타 수정 딕셔너리 (수동 수정할 단어 및 문장)
corrections = {
    "한 국가의 경제 규모름 축정 하는 대표적인 지표이다:.": "한 국가의 경제 규모를 측정하는 대표적인 지표이다.",
    "한 나라의 국민이 국내외에 서 생산한 최종 생산물의 총 가치": "한 나라의 국민이 국내외에서 생산한 최종 생산물의 총 가치",
    "국내충생산에서 해외에서 번 내국인의 소득올 더하고 외 국인이 국내에서 번 소득올 제외": "국내 총생산에서 해외에서 번 내국인의 소득을 더하고 외국인이 국내에서 번 소득을 제외",
    "국민의 종소득 개념올 포함 하여 국제적 비교에 유리하 다.": "국민의 종소득 개념을 포함하여 국제적 비교에 유리하다.",
    "국민종생산에서 감가상각올 제외한 값": "국민 총생산에서 감가상각을 제외한 값",
    "국내총생산에서 감가상각올 제외": "국내총생산에서 감가상각을 제외",
    "국민의 종소득 개념을 포함하여" : "국민의 총소득 개념을 포함하여",
    "국내에서 실질적으로 창출캐 부가가치틀 파악하는데 유용 하다.": "국내에서 실질적으로 창출된 부가가치를 파악하는데 유용하다.",
    "국민종생산에서 감가상각올 제외": "국민총생산에서 감가상각을 제외",
    "국가의 실질적인 경제력올 파악하는데 유용하다.": "국가의 실질적인 경제력을 파악하는데 유용하다."
}

def structure_table_data(table):

    structured_data = {}
    cells = table.meta["cells"]

    categories = ["정의", "계산 방식", "특징"]
    headers = []

    # 테이블의 첫 번째 행에서 각 컬럼의 헤더(제목) 추출
    for cell in cells:
        if cell["row"] == 1:  # 첫 번째 행인 경우
            headers.append(cell["text"].strip())

    num_columns = len(headers)

    for cell in cells:
        row, col = cell["row"], cell["col"]
        text = cell["text"].replace("\n", " ").strip()  # 개행 제거 및 공백 정리

        # '정의', '계산 방식', '특징' 카테고리에 해당하는 데이터만 추출
        if row in [2, 3, 4]:  # 정의, 계산 방식, 특징은 row 2,3,4에 존재
            category = categories[row - 2]


            if col - 1 < num_columns:
                column_name = headers[col - 1]

                if column_name not in structured_data:
                    structured_data[column_name] = {}

                structured_data[column_name][category] = text

    return structured_data

# 도표 내용 줄글로 표시

def format_table_as_sentence(structured_data):
    formatted_sentences = []
    for column_name, content in structured_data.items():
        definition = content.get("정의", "")
        calculation = content.get("계산 방식", "")
        characteristic = content.get("특징", "")

        sentence_parts = [
            f"{column_name}의 정의는 {definition}" if definition else "",
            f"계산 방식은 {calculation}이며" if calculation else "",
            f"특징은 {characteristic}" if characteristic else "",
        ]

        formatted_sentence = " ".join([part for part in sentence_parts if part]) + "."
        formatted_sentences.append(formatted_sentence)

    return formatted_sentences[1:]

# 오타 수정 함수
def correct_text(data, corrections):

    if isinstance(data, list):  # 리스트일 경우, 각 요소에 대해 재귀적으로 처리
        return [correct_text(item, corrections) for item in data]
    elif isinstance(data, str):  # 문자열일 경우, 오타 교정 수행
        for wrong_text, correct_text_value in corrections.items():
            data = data.replace(wrong_text, correct_text_value)
        return data
    else:
        return data  # 문자열이 아닐 경우 원본 그대로 반환

data_df['content'] = data_df.apply(lambda x : " ".join(correct_text(format_table_as_sentence(structure_table_data(x)), corrections)) if (x.region_type == "일반표") and ((x.meta['bounding_box']['y_max'])-(x.meta['bounding_box']['y_min']) > 500) else x.content, axis = 1)
data_df['region_type'] = data_df.apply(lambda x : "평문" if (x.region_type == "일반표") and ((x.meta['bounding_box']['y_max'])-(x.meta['bounding_box']['y_min']) > 500) else x.region_type, axis = 1)

In [43]:
# 필터링된 데이터 저장
data_df.loc[data_df.region_type == "평문"].to_json("output_file.json", orient='records')

In [7]:
%pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-59qdua_m
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-59qdua_m
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2286921 sha256=385b50f3873d7a2d9b38554e5d991f2fa545c4c084ed95d7d7020579815dc52d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ysufmkuy/wheels/1f/3f/64/6d5b2c9ba9cd5aa624676868e8ae8ec684672ce1bcbe20add8
Successfully built pykospacing


##텍스트

In [44]:
text_data = pd.read_json("output_file.json")

In [9]:
%pip install symspellpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 9.9 MB/s eta 0:00:00


In [45]:
import json

# "모두의 말뭉치"에서 교정된 문장 추출
def extract_corrected_sentences(json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        corpus_data = json.load(f)

    extracted_sentences = []
    for doc in corpus_data["document"]:
        for utterance in doc.get("utterance", []):
            if "corrected_form" in utterance:
                extracted_sentences.append(utterance["corrected_form"])  # 교정된 문장만 추출

    return extracted_sentences

# 말뭉치에서 문장 리스트 추출
corpus_texts = extract_corrected_sentences("MXEC2202210100.json")

print(f" 말뭉치에서 추출한 문장 수: {len(corpus_texts)}")

 말뭉치에서 추출한 문장 수: 1129363


In [47]:
from symspellpy import SymSpell, Verbosity
import os

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

#말뭉치에서 추출한 단어 빈도 기반으로 사용자 사전 구축
dictionary_path = "symspell_dictionary.txt"

# 사전 파일 생성
if not os.path.exists(dictionary_path):
    with open(dictionary_path, "w", encoding="utf-8") as f:
        for sentence in corpus_texts:
            words = sentence.split()
            for word in words:
                f.write(f"{word} 1\n")

# SymSpell 사전 로드
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)


True

In [48]:
!pip install datasets

In [49]:
# SymSpell을 활용한 오타 교정
corrected_text_data = []

def symspell(row):
    content = row.content

    # SymSpell로 교정된 문장 찾기 (없으면 원래 문장 유지)
    corrected = sym_spell.lookup(content, Verbosity.CLOSEST, max_edit_distance=2)
    corrected_text = corrected[0].term if corrected else content  # 오타 수정 적용

    return corrected_text

text_data.content = text_data.apply(lambda x : symspell(x), axis = 1)


In [50]:
%pip install torch transformers datasets jsonlines

In [ ]:
import json
import torch
import random
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split

#  KoBART 토크나이저 및 모델 불러오기
MODEL_NAME = "hyunwoongko/kobart"
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)

#  JSON 데이터 로드 및 300,000개 샘플 추출 함수
def load_and_sample_data(json_file, sample_size=300000):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    all_samples = []
    for doc in data["document"]:
        for utterance in doc["utterance"]:
            if "original_form" in utterance and "corrected_form" in utterance:
                all_samples.append((utterance["original_form"], utterance["corrected_form"]))

    # 무작위 샘플링
    sampled_data = random.sample(all_samples, min(sample_size, len(all_samples)))

    inputs, targets = zip(*sampled_data)
    return list(inputs), list(targets)

#  JSON 데이터셋 로드
json_file = "MXEC2202210100.json"  # 모두의 말뭉치 JSON 파일
inputs, targets = load_and_sample_data(json_file)

#  데이터셋을 80% 학습, 20% 검증 데이터로 분리
train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(
    inputs, targets, test_size=0.2, random_state=42
)

#  데이터를 Hugging Face Dataset 형식으로 변환
train_dataset = Dataset.from_dict({"input_text": train_inputs, "target_text": train_targets})
eval_dataset = Dataset.from_dict({"input_text": eval_inputs, "target_text": eval_targets})

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
import os
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast, Trainer, TrainingArguments
from datasets import Dataset

# wandb 비활성화 (환경변수 설정)
os.environ["WANDB_DISABLED"] = "true"

#  GPU 사용 가능 여부 확인 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print(f"✅ GPU 활성화됨: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU를 사용할 수 없습니다. CPU로 진행합니다.")

# KoBART 입력 변환 함수
def preprocess_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(examples["target_text"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 데이터셋 전처리
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# GPU 최적화 설정
torch.cuda.empty_cache()

# 학습 설정
training_args = TrainingArguments(
    output_dir="./kobart_spellcheck",
    evaluation_strategy="epoch",  # 매 epoch마다 평가
    save_strategy="epoch",
    per_device_train_batch_size=32,  # 배치 크기 증가 (학습 속도 향상)
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    report_to="none",  # wandb 사용 안 함
    fp16=True,  # FP16 연산 활성화
)

#  Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

#모델 학습
trainer.train()

✅ GPU 활성화됨: NVIDIA A100-SXM4-40GB


Map:   0%|          | 0/240000 [00:00<?, ? examples/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.017900,0.017118
2,0.012900,0.015716
3,0.009500,0.015333
4,0.006500,0.015864
5,0.004700,0.016641


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


✅ KoBART 모델 학습 및 저장 완료!


In [ ]:
save_path = "/content/kobart_spellcheck"


model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/kobart_spellcheck/tokenizer_config.json',
 '/content/drive/MyDrive/kobart_spellcheck/special_tokens_map.json',
 '/content/drive/MyDrive/kobart_spellcheck/tokenizer.json')

In [51]:
import re
#고쳐지지 않은 부분 수동으로 오타 수정
def fix_particles(row):
    text = row.content
    text = re.sub(r'올\b', '을', text)
    text = re.sub(r'틀\b', '를', text)
    text = re.sub(r'틀\b', '를', text)
    text = re.sub(r'논\b', '는', text)
    text = re.sub(r'않 듣다\b', '않는다', text)
    text = re.sub(r'즉정\b', '측정', text)
    text = re.sub(r'양 논다\b', '않는다', text)
    text = re.sub(r'하분지름', '하는지를', text)
    text = re.sub(r'인플레이선', '인플레이션', text)
    return text

# 조사 수정 적용
text_data.content = text_data.apply(lambda x : fix_particles(x), axis = 1)

In [52]:
text_data

,data_id,page_number,region_type,content,meta
0,page2_class1_b,2,평문,개별 경제주제들이 어떻게 의사결정을 하고 이들이 시장에서 어떻게 상호작용 하는지를 ...,"{'bounding_box': {'x_min': 411.4027404785, 'y_..."
1,page2_class1_a,2,평문,경제 전반에 관한 현상을 연구하는 경제학의 한 분야로 경제주제들의 선택이 집계되어 ...,"{'bounding_box': {'x_min': 412.8612670898, 'y_..."
2,page2_class1_c,2,평문,거시경제 변수 지포로는 국내총생산(GDP) 물가수준 실업률 이자율 무역수지 환율 등...,"{'bounding_box': {'x_min': 408.7577819824, 'y_..."
3,page2_class1_d,2,평문,거시경제 문제로는 인플레이션 실업 경기변동 및 경기순환 경제성장 등이 있다:,"{'bounding_box': {'x_min': 410.277130127, 'y_m..."
4,page3_class1_a,3,평문,1. 모든 선택에는 대가가 있다: 2. 선택의 대가는 그것을 얻기 위해 포기한 그 ...,"{'bounding_box': {'x_min': 402.4910583496, 'y_..."
5,page3_class1_f,3,평문,5 자유거래는 모든 사람을 이롭게 한다:,"{'bounding_box': {'x_min': 412.5721130371, 'y_..."
6,page3_class1_d,3,평문,7. 경우에 따라 정부가 시장성과를 개선활 수 있다:,"{'bounding_box': {'x_min': 413.0425720215, 'y_..."
7,page4_class1_b,4,평문,10개의 경제학의 기본원리 (거시경제학 관련),"{'bounding_box': {'x_min': 270.969329834, 'y_m..."
8,page4_class1_a,4,평문,1. 한 나라의 생활수준은 그 나라의 생산 능력에 달려 있다: 2 통화랑이 지나치게...,"{'bounding_box': {'x_min': 408.7482299805, 'y_..."
9,page4_class1_f,4,평문,1. 한 나라의 생활수준은 그 나라의 생산 능력에 달려 있다:,"{'bounding_box': {'x_min': 409.946105957, 'y_m..."


In [54]:
import json
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 저장된 KoBART 모델 불러오기
model_path = "/content/drive/MyDrive/kobart_spellcheck"# KoBART 저장된 모델 경로
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")


#  KoBART를 사용한 오타 교정 함수
def correct_text(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=256, truncation=True).to("cuda" if torch.cuda.is_available() else "cpu")

    with torch.no_grad():
        output = model.generate(**inputs, max_length=256, num_beams=2, early_stopping=True)

    return tokenizer.decode(output[0], skip_special_tokens=True)

# 데이터프레임에 적용 (오타 교정) - 행 단위 적용
text_data["content"] = text_data.apply(lambda x: correct_text(x["content"]) if isinstance(x["content"], str) else x["content"], axis=1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [55]:
text_data

,data_id,page_number,region_type,content,meta
0,page2_class1_b,2,평문,개별 경제주제들이 어떻게 의사결정을 하고 이들이 시장에서 어떻게 상호작용하는지를 다...,"{'bounding_box': {'x_min': 411.4027404785, 'y_..."
1,page2_class1_a,2,평문,경제 전반에 관한 현상을 연구하는 경제학의 한 분야로 경제주제들의 선택이 집계되어 ...,"{'bounding_box': {'x_min': 412.8612670898, 'y_..."
2,page2_class1_c,2,평문,거시경제 변수 지포로는 국내총생산(GDP) 물가 수준 실업률 이자율 무역수지 환율 ...,"{'bounding_box': {'x_min': 408.7577819824, 'y_..."
3,page2_class1_d,2,평문,"거시경제 문제로는 인플레이션, 실업, 경기 변동 및 경기 순환, 경제성장 등이 있다:","{'bounding_box': {'x_min': 410.277130127, 'y_m..."
4,page3_class1_a,3,평문,1. 모든 선택에는 대가가 있다: 2. 선택의 대가는 그것을 얻기 위해 포기한 그 ...,"{'bounding_box': {'x_min': 402.4910583496, 'y_..."
5,page3_class1_f,3,평문,5 자유 거래는 모든 사람을 이롭게 한다:,"{'bounding_box': {'x_min': 412.5721130371, 'y_..."
6,page3_class1_d,3,평문,7. 경우에 따라 정부가 시장성과를 개선할 수 있다:,"{'bounding_box': {'x_min': 413.0425720215, 'y_..."
7,page4_class1_b,4,평문,10개의 경제학의 기본 원리(거시경제학 관련),"{'bounding_box': {'x_min': 270.969329834, 'y_m..."
8,page4_class1_a,4,평문,1. 한 나라의 생활 수준은 그 나라의 생산 능력에 달려 있다: 2 통화량이 지나치...,"{'bounding_box': {'x_min': 408.7482299805, 'y_..."
9,page4_class1_f,4,평문,1. 한 나라의 생활 수준은 그 나라의 생산 능력에 달려 있다:,"{'bounding_box': {'x_min': 409.946105957, 'y_m..."


In [59]:
# 특정 단어 교정

import re

# 교정할 단어 목록
corrections = {
    "지포로는": "지표로는",
    "다른다:": "다룬다.",
    "앞는": "않는",
    "즉정할": "측정할",
    "시장울": "시장을",
    "변화울을": "변화율을",
    "아분": "아닌",
    "원직적으로": "원칙적으로",
    "7년을": "1년을",
    "꾸즈네즈가": "쿠즈네츠가",
    "뉴필":"뉴딜",
    "자유 거래": "자유거래",
    "경 제학의": "경제학의",
    "실업률 경제활동인구에서": "실업률은 경제활동인구에서",
}

# 특정 단어를 수정하는 함수
def correct_specific_words(row):
    text = row["content"] if isinstance(row["content"], str) else ""  # 문자열이 아닐 경우 빈 문자열로 처리
    for wrong, correct in corrections.items():
        text = re.sub(rf'\b{wrong}\b', correct, text)  # 단어 경계를 고려하여 치환
    return text

# text_data의 "content" 컬럼에 적용
text_data["content"] = text_data.apply(lambda x: correct_specific_words(x), axis=1)


In [57]:
text_data

,data_id,page_number,region_type,content,meta
0,page2_class1_b,2,평문,개별 경제주제들이 어떻게 의사결정을 하고 이들이 시장에서 어떻게 상호작용하는지를 다...,"{'bounding_box': {'x_min': 411.4027404785, 'y_..."
1,page2_class1_a,2,평문,경제 전반에 관한 현상을 연구하는 경제학의 한 분야로 경제주제들의 선택이 집계되어 ...,"{'bounding_box': {'x_min': 412.8612670898, 'y_..."
2,page2_class1_c,2,평문,거시경제 변수 지표로는 국내총생산(GDP) 물가 수준 실업률 이자율 무역수지 환율 ...,"{'bounding_box': {'x_min': 408.7577819824, 'y_..."
3,page2_class1_d,2,평문,"거시경제 문제로는 인플레이션, 실업, 경기 변동 및 경기 순환, 경제성장 등이 있다:","{'bounding_box': {'x_min': 410.277130127, 'y_m..."
4,page3_class1_a,3,평문,1. 모든 선택에는 대가가 있다: 2. 선택의 대가는 그것을 얻기 위해 포기한 그 ...,"{'bounding_box': {'x_min': 402.4910583496, 'y_..."
5,page3_class1_f,3,평문,5 자유 거래는 모든 사람을 이롭게 한다:,"{'bounding_box': {'x_min': 412.5721130371, 'y_..."
6,page3_class1_d,3,평문,7. 경우에 따라 정부가 시장성과를 개선할 수 있다:,"{'bounding_box': {'x_min': 413.0425720215, 'y_..."
7,page4_class1_b,4,평문,10개의 경제학의 기본 원리(거시경제학 관련),"{'bounding_box': {'x_min': 270.969329834, 'y_m..."
8,page4_class1_a,4,평문,1. 한 나라의 생활 수준은 그 나라의 생산 능력에 달려 있다: 2 통화량이 지나치...,"{'bounding_box': {'x_min': 408.7482299805, 'y_..."
9,page4_class1_f,4,평문,1. 한 나라의 생활 수준은 그 나라의 생산 능력에 달려 있다:,"{'bounding_box': {'x_min': 409.946105957, 'y_m..."


In [61]:
#중복 문단 제거

# 삭제할 인덱스 리스트
remove_indices = [5, 6, 8, 15, 18, 22]

# 데이터프레임에서 해당 인덱스 삭제
text_data = text_data.drop(index=remove_indices).reset_index(drop=True)
text_data

,data_id,page_number,region_type,content,meta
0,page2_class1_b,2,평문,개별 경제주제들이 어떻게 의사결정을 하고 이들이 시장에서 어떻게 상호작용하는지를 다...,"{'bounding_box': {'x_min': 411.4027404785, 'y_..."
1,page2_class1_a,2,평문,경제 전반에 관한 현상을 연구하는 경제학의 한 분야로 경제주제들의 선택이 집계되어 ...,"{'bounding_box': {'x_min': 412.8612670898, 'y_..."
2,page2_class1_c,2,평문,거시경제 변수 지표로는 국내총생산(GDP) 물가 수준 실업률 이자율 무역수지 환율 ...,"{'bounding_box': {'x_min': 408.7577819824, 'y_..."
3,page2_class1_d,2,평문,"거시경제 문제로는 인플레이션, 실업, 경기 변동 및 경기 순환, 경제성장 등이 있다:","{'bounding_box': {'x_min': 410.277130127, 'y_m..."
4,page3_class1_a,3,평문,1. 모든 선택에는 대가가 있다: 2. 선택의 대가는 그것을 얻기 위해 포기한 그 ...,"{'bounding_box': {'x_min': 402.4910583496, 'y_..."
5,page4_class1_e,4,평문,2 통화량이 지나치게 증가하면 물가는 상승하다.,"{'bounding_box': {'x_min': 409.9504089355, 'y_..."
6,page5_class1_a,5,평문,1. 생산성 향상 및 신산업 육성 2. 저출산 문제 3. 부동산 시장의 수급 균형과...,"{'bounding_box': {'x_min': 402.7032470703, 'y_..."
7,page6_class1_b,6,평문,주요 거시 변수.,"{'bounding_box': {'x_min': 269.6116027832, 'y_..."
8,page6_class1_h,6,평문,국내 종생산(GDP) 국민소득을 나타내는 지포이다. : 한 나라에서 일정 기간 동안...,"{'bounding_box': {'x_min': 412.5328979492, 'y_..."
9,page6_class1_a,6,평문,인플레이션율 물가의 변화율을 나타내는 지표이다. :,"{'bounding_box': {'x_min': 410.8562011719, 'y_..."


In [32]:
# 수정된 데이터를 JSON 파일로 저장
output_json_path = "/content/updated_text_data.json"
text_data.to_json(output_json_path, orient="records", force_ascii=False, indent=4)

# 수정된 데이터를 CSV 파일로 저장
output_csv_path = "/content/updated_text_data.csv"
text_data.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
